In [ ]:
#importation de la librairie seaborn et pandas
import seaborn as sns
import pandas as pd

#charger et stocker les données de la data frame "sentiment analysis"
data=pd.read_csv("../input/imdb-dataset-sentiment-analysis-in-csv-format/Train.csv")
data.head()

In [ ]:
#réduction du nombre de données pour faciliter et accelerer les calculs (run)
data=data.sample(5000)

In [ ]:
#Question 7

data.label.value_counts()

In [ ]:
data["label"] = data["label"].astype("category")

In [ ]:
data.dtypes

In [ ]:
data.isnull().sum()

In [ ]:
sns.countplot(data.label)

In [ ]:
#afficher la dimension de la dataframe 5000 lignes et 2 colonnes
data.shape

In [ ]:
#changer la chaine de caractères en minuscules de la colonne text
data['text']=data['text'].str.lower()

In [ ]:
#afficher la colonne text
data.text

In [ ]:
#tokenization => but : transformer un texte en une série de tokens individuels. Chaque tocken représente un mot 
#telecharger la fonction word_tokenize de la Bibliothèques nltk.tokenize
from nltk.tokenize import word_tokenize

In [ ]:
#importer et afficher les ponctuations
import string
string.punctuation

In [ ]:
#fonction qui supprime la ponctuation dans la colonne text
def remove_punct(text):
    text_tok = word_tokenize(text)
    l=[]
    for word in text_tok: 
        if not word in string.punctuation:
            l.append(word)
           
    resultat=" ".join(l)  
    return resultat

In [ ]:
#application de la fonction sur la colonne text
data['text']=data.text.apply(remove_punct)

In [ ]:
#afficher la colonne text
data.text

In [ ]:
from nltk.corpus import stopwords
#Télècharger les stopwords
stop=set(stopwords.words('english'))


In [ ]:
#fonction qui supprime les stopwords dans la colonne text
def remove_stopword(text):
    text_tok = word_tokenize(text)
    l = []
    for a in text_tok:
        if not a in stop:
            l.append(a)
            
    resultat = " ".join(l)
    return resultat   

In [ ]:
#application de la fonction sur la colonne text
data.text=data.text.apply(remove_stopword)

In [ ]:
#Lemmatization: ramené un mot à sa forme la plus simple 
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [ ]:
#fonction lemmatization sur la colonne text
def lemm(text):
    text_tok = word_tokenize(text)
    l = []
    for word in text_tok:
        l.append(lemmatizer.lemmatize(word))
            
    resultat = " ".join(l)
    return resultat 

In [ ]:
#fonction lemmatization sur la colonne text
data.text=data.text.apply(lemm)

In [ ]:
#véctoriser par la méthode : bag of words
corpus=data['text'].values


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bw_vect = CountVectorizer()
# tokenize et construire le vocabulaire
bw_fit=bw_vect.fit(corpus)
# vectoriser les mots
bw_corpus = bw_fit.transform(corpus)

In [ ]:
#afficher la dimension 5000 lignes et 35152 colonnes
bw_corpus.shape

In [ ]:
#affichage de la véctorisation  BW
cv_data=pd.DataFrame(bw_corpus.toarray(),columns=bw_fit.get_feature_names())
cv_data.head(7)

In [ ]:
#véctoriser par la méthode : TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#Initialiser les paramètres du vectoriseur
tf_vect = TfidfVectorizer(max_features=5000)
#Apprendre le vocabulaire du vectoriseur basé sur le paramètre initialisé
tfidf_fit=tf_vect.fit(corpus)
#Vectoriser le corpus
tfidf_corpus= tfidf_fit.transform(corpus)

In [ ]:
#affichage de la véctorisation  TFIDF
tfidf_data=pd.DataFrame(tfidf_corpus.toarray(),columns=tfidf_fit.get_feature_names())
tfidf_data.head()

In [ ]:
#préparation des données pour l'algorithme d'apprentissage et pour le test : BW
#attribution des données pour Y et X ref au tableau BW
#sauvegarde des commentaires vectorisés dans une variable "X" et sauvegarder la colonne "label" dans une variable "Y"
from sklearn.model_selection import train_test_split
Y=data.label
X=cv_data
# fonction train_test_split : permet de décomposer le jeu de données en 2 groupes (données apprentissage / données tests)
# Split train / test data :
X_train_bw, X_test_bw, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [ ]:
#Construire l'arbre de décision qui fait le lien entre Y_train et X_train
from sklearn import tree
tree_model = tree.DecisionTreeClassifier()
#tree_model = tree.DecisionTreeClassifier(max_depth = 2)
tree_model=tree_model.fit(X_train_bw, Y_train)

In [ ]:
#le nombre de commentaires négatifs et positifs
Y_train.value_counts()

In [ ]:
names = ['1', '0']

In [ ]:
#en utilisant l'arbre de décision, deviner si chaque commentaire de la variable  X_test est positif ou négatif. Sauvegarder la réponse dans la variable Y_predict 
Y_predict_bw=tree_model.predict(X_test_bw)

In [ ]:
# Plot Matrice de confusion :
from sklearn.metrics import accuracy_score, confusion_matrix 
mat = confusion_matrix(Y_predict_bw, Y_test)

In [ ]:
#Permet de définir la fonction plt
from matplotlib import pyplot as plt

In [ ]:
#heatmap : dégradé de couleurs qui représentent les différentes valeurs
sns.heatmap(mat, annot=True,  xticklabels=names, yticklabels=names)
#nommer les axes du graphe
plt.xlabel('Test')
plt.ylabel('Predicted_bw')

In [ ]:
# Commentaire positif = 1
#5.5x10^2 représentent les commentaires positifs et 2,3x10^2 représentent des commentaires comptés positifs alors qu'ils sont normalement négatifs (erreurs)
# % erreurs de commentaires positifs = 29,5 %
#Commentaire négatif = 0
#5x10^2 représentent les commentaires négatifs et 2,3x10^2 représentent des commentaires comptés négatifs alors qu'ils sont normalement positifs (erreurs)
# % erreurs de commentaires négatifs = 29,5 %

In [ ]:
#préparation des données pour l'algorithme d'apprentissage et pour le test : TFIDF
#attribution des données pour Y et X ref au tableau TFIDF
#sauvegarde des commentaires vectorisés dans une variable "X" et sauvegarder la colonne "label" dans une variable "Y"
from sklearn.model_selection import train_test_split
Y=data.label
X=tfidf_data
# fonction train_test_split : permet de décomposer le jeu de données en 2 groupes (données apprentissage / données tests)
# Split train / test data :
X_train_tfidf, X_test_tfidf, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [ ]:
#Construire l'arbre de décision qui fait le lien entre Y_train et X_train
from sklearn import tree
tree_model = tree.DecisionTreeClassifier()
#tree_model = tree.DecisionTreeClassifier(max_depth = 2)
tree_model=tree_model.fit(X_train_tfidf, Y_train)

In [ ]:
#le nombre de commentaires négatifs et positifs
Y_train.value_counts()

In [ ]:
#en utilisant l'arbre de décision, deviner si chaque commentaire de la variable  X_test est positif ou négatif. Sauvegarder la réponse dans la variable Y_predict
Y_predict_tfidf=tree_model.predict(X_test_tfidf)

In [ ]:
# Plot Matrice de confusion :
from sklearn.metrics import accuracy_score, confusion_matrix 
mat = confusion_matrix(Y_predict_tfidf, Y_test)

In [ ]:
#heatmap : dégradé de couleurs qui représentent les différentes valeurs
sns.heatmap(mat, annot=True,  xticklabels=names, yticklabels=names)
#nommer les axes du graphe
plt.xlabel('Test')
plt.ylabel('Predicted_tfidf')

In [ ]:
# Commentaire positif = 1
#5.4x10^2 représentent les commentaires positifs et 2,3x10^2 représentent des commentaires comptés positifs alors qu'ils sont normalement négatifs (erreurs)
# % erreurs de commentaires positifs = 29,8 %
#Commentaire négatif = 0
#5.1x10^2 représentent les commentaires négatifs et 2,2x10^2 représentent des commentaires comptés négatifs alors qu'ils sont normalement positifs (erreurs)
# % erreurs de commentaires négatifs = 30,1 %

In [ ]:
#gridsearch sur tfidf
#gridsearch : optimisation qui va nous permettre de tester des séries de paramètres pour en déduire le meilleur paramétrage 
#découpage du dataset en k échantillon
#importation 
from sklearn.model_selection import GridSearchCV
import numpy as np

In [ ]:
depths = np.arange(10, 40,5)
param_grid = [{'max_depth':depths}]
grid_tree= GridSearchCV(estimator=tree.DecisionTreeClassifier(),param_grid=param_grid,scoring='accuracy',cv=10)
grid_tree.fit(X_train_tfidf, Y_train)
best_model_tree = grid_tree.best_estimator_
Y_grid=best_model_tree.predict(X_test_tfidf)

# Accuracy:
accuracy_score(Y_test, Y_grid)
grid_tree.best_params_

In [ ]:
#Construction de la forêt d'arbre méthode bw
from sklearn.ensemble import RandomForestClassifier

In [ ]:
Rf_model = RandomForestClassifier()
Rf_model=Rf_model.fit(X_train_bw, Y_train)

In [ ]:
#en utilisant l'arbre de décision, deviner si chaque commentaire de la variable  X_test est positif ou négatif. Sauvegarder la réponse dans la variable Y_predict
Y_predict=Rf_model.predict(X_test_bw)

In [ ]:
mat = confusion_matrix(Y_predict, Y_test)
#heatmap : dégradé de couleurs qui représentent les différentes valeurs
sns.heatmap(mat, annot=True,  xticklabels=names, yticklabels=names)
#nommer les axes du graphe
plt.xlabel('Test')
plt.ylabel('Predicted_bw')

In [ ]:
# On obtient beaucoup moins d'erreurs
#6.4x10^2 représentent les commentaires positifs et 1,3x10^2 représentent des commentaires comptés positifs alors qu'ils sont normalement négatifs (erreurs)
# % erreurs de commentaires positifs = 16,6 %
#6x10^2 représentent les commentaires négatifs et 1,3x10^2 représentent des commentaires comptés négatifs alors qu'ils sont normalement positifs (erreurs)
# % erreurs de commentaires négatifs = 17,8 %

In [ ]:
#Construction de la forêt d'arbre méthode tfidf
from sklearn.ensemble import RandomForestClassifier

In [ ]:
Rf_model = RandomForestClassifier()
Rf_model=Rf_model.fit(X_train_tfidf, Y_train)

In [ ]:
#en utilisant l'arbre de décision, deviner si chaque commentaire de la variable  X_test est positif ou négatif. Sauvegarder la réponse dans la variable Y_predict
Y_predict=Rf_model.predict(X_test_tfidf)

In [ ]:
mat = confusion_matrix(Y_predict, Y_test)
#heatmap : dégradé de couleurs qui représentent les différentes valeurs
sns.heatmap(mat, annot=True,  xticklabels=names, yticklabels=names)
#nommer les axes du graphe
plt.xlabel('Test')
plt.ylabel('Predicted_tfidf')

In [ ]:
# On obtient beaucoup moins d'erreurs
#6.5x10^2 représentent les commentaires positifs et 1,2x10^2 représentent des commentaires comptés positifs alors qu'ils sont normalement négatifs (erreurs)
# % erreurs de commentaires positifs = 15,6 %
#6x10^2 représentent les commentaires négatifs et 1,3x10^2 représentent des commentaires comptés négatifs alors qu'ils sont normalement positifs (erreurs)
# % erreurs de commentaires négatifs = 17,8 %

In [ ]:
#fonction classer 
def classer(commentaire):
    
    text=text.lower()
    text=text.replace('covid-19','coronavirus')
    text=remove_punct(text)
    text=remove_stopword(text)
    text=lemm(text)
    tfidf_text=tfidf_fit.transform([text])
    
    cm=cosine_similarity(tfidf_text, tfidf_corpus)
    pos=np.argmax(cm[0])
    data.answers[pos]
    
    return resultat

In [ ]:
reponse('does the weather have any impact on the propgation of the covid-19')

In [ ]:
#
while True:
    text = str(input("Input: "))
    if text== "exit":
        print("Response: Exiting.....")
        break
    print("Response:",classer(text))